In [1]:
import pandas as pd
import yaml
import os
import sys
import numpy as np
import glob 
import pickle
sys.path.append('./../../')
sys.path.append('./..')
import inspect
import matplotlib.pyplot as plt
import sys
import time
import yaml
import time
from collections import OrderedDict

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import auc
import glob

In [2]:
from src.ape import tf_model_ape_1
from src.data_fetcher import data_fetcher


/home/ddatta/anaconda3/envs/AD_v2/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ddatta/anaconda3/envs/AD_v2/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ddatta/anaconda3/envs/AD_v2/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ddatta/anaconda3/envs/AD_v2/lib/python3.6/s

In [3]:
CONFIG = None
OP_DIR = None
DATA_DIR = None
domain_dims = None
SAVE_DIR = None
CONFIG_FILE = 'config_1.yaml'
domain_dims = None

# ------------------------------------- #
def get_domain_dims(dd_file_path):
    with open(dd_file_path, 'rb') as fh:
        domain_dims = pickle.load(fh)
    _tmpDF = pd.DataFrame.from_dict(domain_dims, orient='index')
    _tmpDF = _tmpDF.reset_index()
    _tmpDF = _tmpDF.rename(columns={'index': 'domain'})
    _tmpDF = _tmpDF.sort_values(by=['domain'])
    res = {k: v for k, v in zip(_tmpDF['domain'], _tmpDF[0])}
    return res
# ------------------------------------- #
# Set up config & global vars
# ------------------------------------- #
def setup():
    global SAVE_DIR
    global DIR
    global CONFIG
    global OP_DIR
    global DATA_DIR
    global domain_dims
   
    with open(CONFIG_FILE) as f:
        CONFIG = yaml.safe_load(f)
  
    SAVE_DIR = CONFIG['SAVE_DIR']
    DIR = CONFIG['DIR']
        
    OP_DIR = CONFIG['OP_DIR']
    DATA_DIR = CONFIG['DATA_DIR']
    
    if not os.path.exists(SAVE_DIR):
        os.mkdir(SAVE_DIR)
    SAVE_DIR = os.path.join(SAVE_DIR, DIR)
 
    if not os.path.exists(SAVE_DIR):
        os.mkdir(SAVE_DIR)

    if not os.path.exists(OP_DIR):
        os.mkdir(OP_DIR)

    OP_DIR = os.path.join(OP_DIR, DIR)
    if not os.path.exists(OP_DIR):
        os.mkdir(OP_DIR)
        
    tf_model_ape_1._DIR = DIR
    tf_model_ape_1.OP_DIR = OP_DIR
    if not os.path.exists(OP_DIR):
        os.mkdir(OP_DIR)
        
    
    dd_file_path = os.path.join(DATA_DIR, DIR, 'domain_dims.pkl')
    domain_dims = get_domain_dims(dd_file_path)
 
    print('DATA_DIR : ' ,DATA_DIR)
    print('OP_DIR :', OP_DIR)
    print('SAVE_DIR :', SAVE_DIR)
    
setup()

DATA_DIR :  ../../generated_data_v1
OP_DIR : output/us_import1
SAVE_DIR : save_dir/us_import1


## --------------------------------- 

This function gets the testing data
Set :
1. Anomaly percentage to be 20 % of data: So 1:4 ratio
2. Why type 2 ? : Clusters
3. what are the configs ? 'test_anomaly_perc'

## ----------------------------------


In [4]:
def get_testing_data(anomaly_type = 2):
    global DIR
    global DATA_DIR
    global CONFIG
    
    p =  int(CONFIG['test_anomaly_perc'])
    _, _, _, _, test_x ,test_idList, anomaly_x, anomaly_idList = data_fetcher.get_data_APE(
        DATA_DIR,
        DIR,
        anomaly_type=2
    )
    print(len(anomaly_idList))
    print(len(test_idList))
    reqd_anom_count = int(p/(100-p) * len(test_idList))
    
    anomaly_idList = anomaly_idList[:reqd_anom_count]
    anomaly_x = anomaly_x[:reqd_anom_count]
    
    return  test_x ,test_idList, anomaly_idList, anomaly_x
    

In [5]:

def run_APE():
    global DIR
    global OP_DIR
    global SAVE_DIR
    global DATA_DIR
    global CONFIG
    global domain_dims
    
    MODEL_NAME = 'model_APE'
    
    # ------------ #
    
    checkpoint_dir = os.path.join(SAVE_DIR,'checkpoints')
    if not os.path.exists(checkpoint_dir):
        os.mkdir(checkpoint_dir)

    _DATA_DIR = CONFIG['DATA_DIR']
    train_x_pos, train_x_neg, APE_term_2, APE_term_4, _, _,  _,_ = data_fetcher.get_data_APE(
        _DATA_DIR,
        DIR
    )
    
    print(train_x_pos.shape, train_x_neg.shape, APE_term_2.shape, APE_term_4.shape)
    
    neg_samples = train_x_neg.shape[1]
    start_time = time.time()
    num_domains = len(domain_dims)
    inp_dims = list(domain_dims.values())

    print('Number of domains ', num_domains )
    print(' domains ', inp_dims)
    ape_model_obj = tf_model_ape_1.model_ape_1(MODEL_NAME)
    
    ape_model_obj.set_model_params(
        num_entities = num_domains,
        inp_dims = inp_dims,
        neg_samples = neg_samples,
        batch_size = CONFIG[DIR]['ape_batchsize'],
        num_epochs = CONFIG[DIR]['num_epochs'],
        lr = CONFIG[DIR]['ape_learning_rate'],
        chkpt_dir = checkpoint_dir
    )

    _emb_size = int(CONFIG[DIR]['ape_embed_size'])
    ape_model_obj.set_hyper_parameters(
        emb_dims=[_emb_size],
        use_bias=[True, False]
    )

    _use_pretrained = CONFIG[DIR]['ape_use_pretrained']

    if _use_pretrained is False:
        print(' Building and training model ')
        ape_model_obj.build_model()
        ape_model_obj.train_model(
            train_x_pos,
            train_x_neg,
            APE_term_2,
            APE_term_4
        )
    
    
    bounds = []
    training_pos_scores = model_obj.inference(
        train_x_pos
    )
    
    training_pos_scores = [_[0] for _ in training_pos_scores]

    train_noise = np.reshape(train_x_neg, [-1, train_x_pos.shape[-1]])
    training_noise_scores = model_obj.inference(
        train_noise
    )
    training_noise_scores = [_[0] for _ in training_noise_scores]

    bounds.append(min(training_noise_scores))
    bounds.append(max(training_pos_scores))

    test_x ,test_idList, anomaly_idList, anomaly_x = get_testing_data(anomaly_type = 2)
    

    '''
    join the normal data + anomaly data
    join the normal data id +  anomaly data id 
    Maintain order
    '''
    test_normal_ids = test_idList
    test_anomaly_ids = anomaly_idList
    test_ids = list(np.hstack(
        [test_normal_ids,
         test_anomaly_ids]
    ))
    print (' Len of test_ids ', len(test_ids))
    test_normal_data = test_x
    test_anomaly_data = anomaly_x
    test_data_x = np.vstack([
        test_normal_data,
        test_anomaly_data
    ])

    # ---------- #

    print('Length of test data',test_data_x.shape)
    res = model_obj.inference(
        test_data_x
    )

    test_ids = list(test_ids)
    print('Length of results ', len(res))


    res = list(res)
    _id_score_dict = {
        id: _res for id, _res in zip(test_ids, res)
    }

    '''
    sort by ascending 
    since lower likelihood means anomalous
    '''
    tmp = sorted(
        _id_score_dict.items(),
        key=operator.itemgetter(1)
    )
    sorted_id_score_dict = OrderedDict()

    for e in tmp:
        sorted_id_score_dict[e[0]] = e[1][0]
        recall, precison = eval.precision_recall_curve(
            sorted_id_score_dict,
            anomaly_id_list=test_anomaly_ids
        )

        recall_str = ','.join([str(_) for _ in recall])
        precision_str = ','.join([str(_) for _ in precison])
    print( 'AUC ::' , auc(precison,recall))
    return 




In [6]:
run_APE()

(140382, 8) (140382, 24, 8) (140382, 1) (140382, 24, 1)
Number of domains  8
 domains  [548, 5113, 95, 238, 64, 113, 116, 6193]
save_dir/us_import1/checkpoints/model_APE_12_k_24_frozen.pb
 Building and training model 




Epoch ::  1
Epoch ::  1 Batch : 0  Loss : -251.0401
Epoch ::  1 Batch : 200  Loss : -197.1187
Epoch ::  1 Batch : 400  Loss : -196.28271
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 18 variables.
INFO:tensorflow:Converted 18 variables to const ops.
Saving file.  save_dir/us_import1/checkpoints/model_APE_12_k_24_frozen.pb
